# MTH 496 HW3 

Due date: 5pm, Nov 29, 2022

Katie Pocock

In [119]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math as mth

**&#9989; **Problem 1 (40pts)****.  The Digits datasets are given. Classify this dataset using ANN models with different layer configurations. Note that the layer configuration must be an adjustable parameter: the number of layers and the number of neurons in each layer can be chosen arbitrarily (within reasonable limits). Present your results with the following layer configurations:
- One hidden layer with 200 neurons.
- Two hidden layers: each has 100 neur\ons.
- Five hidden layers with 50,100,100,100,50 neurons, respectively.

Two hidden layers : (did it the first by basing it off of the ann code.)

In [120]:
class StandardScaler:
    def __init__(self):
        pass
    def fit(self,X_train): ##fit X_train, calculate mean, std of X_train
        self.u = X_train.mean(axis=0)
        self.s = X_train.std(axis=0)
    def transform(self,X):#apply normlization on X_train or X_test
        X_norm = (X-self.u)/(self.s+1e-6)## z = (x-u)/s, check: will self.s = 0 ?
        return X_norm
    
class one_hot_encoder:
    def __init__(self):
        pass
    def fit(self,y_train):
    #fit y_train, find set{y_train} and assign elements of this set with one-hot-vectors.
       label_set = set(y_train)
    #assign index to labels:
       self.label_list = list(label_set)
    
    def transform(self,y): #apply one-hot-encoding to y_train or y_test
        #the shape of output matrix is (y.shape[0],len(self.label_list))
        ohe_matrix = np.zeros((y.shape[0],len(self.label_list)))
        for label,vector in zip(y,ohe_matrix):
            #find index of label and add 1 to vector[index]
            index = self.label_list.index(label)
            vector[index]+=1
        return ohe_matrix.astype(int)
    
    
    
def read_dataset(feature_file,label_file):
    df_X = pd.read_csv(feature_file)
    df_y = pd.read_csv(label_file)
    X = df_X.values #convert values in dataframe to numpy 2-D array
    y = df_y.values.reshape(-1)   #convert values in dataframe to numpy 1-D array
    return X,y



def normalize_features(X_train,X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    return scaler.transform(X_train),scaler.transform(X_test)




def ohe(y_train,y_test):
    ohencoder = one_hot_encoder()
    ohencoder.fit(y_train)
    return ohencoder.transform(y_train),ohencoder.transform(y_test)




def softmax(z):
    exp_value = np.exp(z-np.amax(z, axis=1, keepdims=True)) # for stablility
    # keepdims = True means that the output's dimension is the same as of z
    softmax_scores = exp_value / np.sum(exp_value, axis=1, keepdims=True)
    return softmax_scores



def accuracy(ypred, yexact):
    p = np.array(ypred == yexact, dtype = int)
    return np.sum(p)/float(len(yexact))

In [121]:
class twolayer_NN:
    def __init__(self, X, y, hidden_layer_nn_0=100, hidden_layer_nn_1=100, lr=0.01):
        self.X = X # features
        self.y = y # labels (targets) in one-hot-encoder
        self.lr = lr # learning rate
        # Initialize weights
        self.nn = X.shape[1] # number of neurons in the input layer

        self.Wp = np.random.randn(self.nn, hidden_layer_nn_0) / np.sqrt(self.nn)
        self.bp = np.zeros((1, hidden_layer_nn_0)) 
        
        self.W0 = np.random.randn(hidden_layer_nn_0, hidden_layer_nn_1) / np.sqrt(hidden_layer_nn_0)
        self.b0 = np.zeros((1, hidden_layer_nn_1)) 
        
        self.output_layer_nn = y.shape[1]
        self.Wq = np.random.randn(hidden_layer_nn_1, self.output_layer_nn) / np.sqrt(hidden_layer_nn_1)
        self.bq = np.zeros((1, self.output_layer_nn))      
        
    def feed_forward(self):
        # hidden layer
        ## z_0 = XW_p + b_p
        self.z0 = np.dot(self.X, self.Wp) + self.bp
        ## activation function :  f_0 = \tanh(z_0)
        self.f0 = np.tanh(self.z0)
        ## z_1 = f_0W_0 + b_0
        self.z1 = np.dot(self.f0, self.W0) + self.b0
        ## activation function :  f_1 = \tanh(z_1)
        self.f1 = np.tanh(self.z1)
        
        # output layer
        ## z_q = f_1W_q + b_q
        self.zq = np.dot(self.f1, self.Wq) + self.bq   
        #\hat{y} = softmax}(z_q)$
        self.y_hat = softmax(self.zq)
        
    def back_propagation(self):
        # $d_3 = \hat{y}-y$
        dq = self.y_hat - self.y
        # d_2 = (1-f^2_2)*(\hat{y}-y)W_3^T
        d0 = (1-self.f1*self.f1)*(dq.dot((self.Wq).T))
        dp = (1-self.f0*self.f0)*(d0.dot((self.W0).T))
        
        # dL/dWq = f_1^T d3
        dWq = np.dot(self.f1.T, dq)
        # dL/dbq = sum(dq,axis=0)
        dbq = np.sum(dq, axis=0, keepdims=True)
        
        # dL/dW0 = f_0^T d_0
        dW0 = np.dot(self.f0.T, d0)
        # dL/b_0 = sum(d_0,axis=0)
        db0 = np.sum(d0, axis=0, keepdims=True)
        # axis =0 : sum along the vertical axis

        # dL/dW_1} = x^T d_1
        dWp = np.dot((self.X).T, dp)
        # dL/db_1 = d_1.axis=0
        dbp = np.sum(dp, axis=0, keepdims=True)
        
        # Update the gradident descent
        self.Wp = self.Wp - self.lr * dWp
        self.bp = self.bp - self.lr * dbp
        self.W0 = self.W0 - self.lr * dW0
        self.b0 = self.b0 - self.lr * db0
        self.Wq = self.Wq - self.lr * dWq
        self.bq = self.bq - self.lr * dbq
        
    def cross_entropy_loss(self):
        #  $L = -\sum_n\sum_{i\in C} y_{n, i}\log(\hat{y}_{n, i})$
        # calculate y_hat
        self.feed_forward()
        self.loss = -np.sum(self.y*np.log(self.y_hat + 1e-6))
        
    def predict(self, X_test):
        # Use feed forward to calculat y_hat_test
        # hidden layer
        ## z_p = XW_p + b_p
        z0 = np.dot(X_test, self.Wp) + self.bp
        ## activation function :  f_0 = \tanh(z_0)
        f0 = np.tanh(z0)
        ## z_1 = f_0W_0 + b_0
        z1 = np.dot(f0, self.W0) + self.b0
        f1 = np.tanh(z1)
        # output layer
        ## z_q = f_1W_q + b_q
        zq = np.dot(f1, self.Wq) + self.bq    
        #\hat{y} = softmax(z_q)$
        y_hat_test = softmax(zq)
        # the rest is similar to the logistic regression
        labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        num_test_samples = X_test.shape[0]
        # find which index gives us the highest probability
        ypred = np.zeros(num_test_samples, dtype=int) 
        for i in range(num_test_samples):
            ypred[i] = labels[np.argmax(y_hat_test[i,:])]
        return ypred

In [122]:
X_train, y_train = read_dataset('Digits_X_train.csv', 'Digits_y_train.csv')
X_test, y_test = read_dataset('Digits_X_test.csv', 'Digits_y_test.csv')


In [123]:
# 
my2NN = twolayer_NN(X_train_norm, y_train_ohe, hidden_layer_nn_0=300, hidden_layer_nn_1=100, lr=0.001)  
epoch_num = 200
for i in range(epoch_num):
    my2NN.feed_forward()
    my2NN.back_propagation()
    my2NN.cross_entropy_loss()
    if ((i+1)%20 == 0):
        print('epoch = %d, current loss = %.5f' % (i+1, my2NN.loss))         
        
y_pred = my2NN.predict(X_test_norm)
print('Accuracy of our model ', accuracy(y_pred, y_test.ravel()))

epoch = 20, current loss = 149.46214
epoch = 40, current loss = 42.32578
epoch = 60, current loss = 20.79385
epoch = 80, current loss = 13.12941
epoch = 100, current loss = 9.39530
epoch = 120, current loss = 7.23351
epoch = 140, current loss = 5.83950
epoch = 160, current loss = 4.87256
epoch = 180, current loss = 4.16581
epoch = 200, current loss = 3.62850
Accuracy of our model  0.0


<ipython-input-120-33d6251bed83>:65: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  p = np.array(ypred == yexact, dtype = int)


In [166]:
#help classes and functions (same as ANN).
##one-hot-encoding
class one_hot_encoder:
    def __init__(self):
        pass
    def fit(self,y_train):
    #fit y_train, find set{y_train} and assign elements of this set with one-hot-vectors.
       label_set = set(y_train)
    #assign index to labels:
       self.label_list = list(label_set)
    
    def transform(self,y): #apply one-hot-encoding to y_train or y_test
        #the shape of output matrix is (y.shape[0],len(self.label_list))
        ohe_matrix = np.zeros((y.shape[0],len(self.label_list)))
        for label,vector in zip(y,ohe_matrix):
            #find index of label and add 1 to vector[index]
            index = self.label_list.index(label)
            vector[index]+=1
        return ohe_matrix.astype(int)
    
def ohe(y_train,y_test):
    ohencoder = one_hot_encoder()
    ohencoder.fit(y_train)
    return ohencoder.transform(y_train),ohencoder.transform(y_test)
### normalize feature
class StandardScaler:
    def __init__(self):
        pass
    def fit(self,X_train): ##fit X_train, calculate mean, std of X_train
        self.u = X_train.mean(axis=0)
        self.s = X_train.std(axis=0)
    def transform(self,X):#apply normlization on X_train or X_test
        X_norm = (X-self.u)/(self.s+1e-6)## z = (x-u)/s, check: will self.s = 0 ?
        return X_norm
    
def normalize_features(X_train,X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    return scaler.transform(X_train),scaler.transform(X_test)    
## scoring function
def accuracy(ypred, yexact):
    p = np.array(ypred == yexact, dtype = int)
    return np.sum(p)/float(len(yexact))
## read dataset
def read_dataset(feature_file,label_file):
    df_X = pd.read_csv(feature_file)
    df_y = pd.read_csv(label_file)
    X = df_X.values #convert values in dataframe to numpy 2-D array
    y = df_y.values.reshape(-1)   #convert values in dataframe to numpy 1-D array
    return X,y

def to_tensor(X):
    #2d array to 4d array
    m_samples, n_features=X.shape
    return X.reshape(m_samples,1,int(np.sqrt(n_features)),int(np.sqrt(n_features)))# Minist dataset features 874=28*28

#convolutional operation
def conv(x, k):
    x_row, x_col = x.shape # data shape
    k_row, k_col = k.shape # kernel shape 
    
    result_row, result_col = x_row - k_row + 1, x_col - k_col + 1 #stride = 1
    result = np.empty((result_row, result_col))
    for i in range(result_row):
        for j in range(result_col):
            #summation of point-wise product of submatrix with kernel
            sub = x[i : i + k_row, j : j + k_col]
            result[i,j] = np.sum(sub * k)
    return result

#padding x with size1 rows/ size2 columns of zeros.
def padding(x, size1,size2):
    cur_r, cur_w = x.shape[0], x.shape[1]
    new_r = cur_r + size1 * 2
    new_w = cur_w + size2 * 2
    result = np.zeros((new_r, new_w))
    result[size1:cur_r + size1, size2:cur_w+size2] = x
    return result

def rot180(w):
#rotate matrix for 180 degree:
#array([[ 0,  1,  2,  3],
#       [ 4,  5,  6,  7],
#       [ 8,  9, 10, 11]])
# to
#array([[11, 10,  9,  8],
#       [ 7,  6,  5,  4],
#       [ 3,  2,  1,  0]])
    m,n = w.shape 
    w180 = w.flatten()[::-1].reshape(m,n)
    return w180

class ConvLayer1:
    def __init__(self,lr=0.001):
        # in this example, in_channel = 1, out_channel = 2, kernel shape is (3,3)
        self.w = np.random.randn(1, 1, 5,5)/np.sqrt(5)
        self.b = np.zeros((2)) 
        self.lr = lr
    
    def forward(self, in_data):      
        self.input = in_data 
        
        m_samples, in_channel, in_row, in_col = in_data.shape
        out_channel = 1
        # the kernel shape is (5,5)
        out_row = in_row - 4
        out_col = in_col - 4
        
        self.output = np.zeros((m_samples, out_channel, out_row, out_col))
        #You have to iterate in_channel if in_channel>1.
        for m in range(m_samples):
            for o in range(out_channel):
                self.output[m, o,:,:] += conv(in_data[m, 0,:,:], self.w[0, o,:,:])
                self.output[m, o,:,:] += self.b[o]# add bias for each output
        return self.output
    
    def backward(self,dz):
        m_samples, out_channel, out_row, out_col = dz.shape #dz has the same shape as the output

        # gradient of b has nothing to do with conv function; similar as ann's b term.
        db = np.sum(dz,axis=(0,2,3))
        
        
        #calculate dw to update current layers' gradient and dx for previous layer to update gradient
        dw = np.zeros_like(self.w)
        dx = np.zeros_like(self.input)
        
        #dL/dw  = sum_m con(x,dz). You have to iterate in_channel if in_channel>1.
        for m in range(m_samples):
            for o in range(out_channel):
                dw[0, o,:,:] += conv(self.input[m,0,:,:], dz[m,o,:,:])
                  
        #dL/dx = sum_i conv(pad(dz),rot180(w)).You have to iterate in_channel if in_channel>1.
        for m in range(m_samples):
            for o in range(out_channel):
                dz_padded = padding(dz[m,o,:,:],4,4)
                w_rot = rot180(self.w[0,o,:,:])
                dx[m, 0,  :,:] += conv(dz_padded,w_rot)

        # update gradient
        self.w = self.w - self.lr*dw
        self.b = self.b - self.lr*db
        ##return dx for previous layer to calculate gradient    
        return dx
  

class ConvLayer2:
    def __init__(self,lr=0.001):
        # in this example, in_channel = 1, out_channel = 2, kernel shape is (3,3)
        self.w = np.random.randn(1, 2, 3, 3)/np.sqrt(3)
        self.b = np.zeros((2)) 
        self.lr = lr
    
    def forward(self, in_data):      
        self.input = in_data 
        
        m_samples, in_channel, in_row, in_col = in_data.shape
        out_channel = 2
        # the kernel shape is (3,3)
        out_row = in_row - 2
        out_col = in_col - 2
        
        self.output = np.zeros((m_samples, out_channel, out_row, out_col))
        #You have to iterate in_channel if in_channel>1.
        for m in range(m_samples):
            for o in range(out_channel):
                self.output[m, o,:,:] += conv(in_data[m, 0,:,:], self.w[0, o,:,:])
                self.output[m, o,:,:] += self.b[o]# add bias for each output
        return self.output
    
    def backward(self,dz):
        m_samples, out_channel, out_row, out_col = dz.shape #dz has the same shape as the output

        # gradient of b has nothing to do with conv function; similar as ann's b term.
        db = np.sum(dz,axis=(0,2,3))
        
        
        #calculate dw to update current layers' gradient and dx for previous layer to update gradient
        dw = np.zeros_like(self.w)
        dx = np.zeros_like(self.input)
        
        #dL/dw  = sum_m con(x,dz). You have to iterate in_channel if in_channel>1.
        for m in range(m_samples):
            for o in range(out_channel):
                dw[0, o,:,:] += conv(self.input[m,0,:,:], dz[m,o,:,:])
                  
        #dL/dx = sum_i conv(pad(dz),rot180(w)).You have to iterate in_channel if in_channel>1.
        for m in range(m_samples):
            for o in range(out_channel):
                dz_padded = padding(dz[m,o,:,:],2,2)
                w_rot = rot180(self.w[0,o,:,:])
                dx[m, 0,  :,:] += conv(dz_padded,w_rot)

        # update gradient
        self.w = self.w - self.lr*dw
        self.b = self.b - self.lr*db
        ##return dx for previous layer to calculate gradient    
        return dx
    

    
class MaxPoolingLayer:
    def __init__(self, pooling = (2,2), strides = (2,2)):
        self.pooling = pooling # 
        self.strides = strides 

    def forward(self, in_data):   
        m_samples, in_channel, in_row, in_col = in_data.shape
        s0 = self.strides[0]
        s1 = self.strides[1]
        out_row = (in_row - self.pooling[0]) // s0 + 1
        out_col = (in_col - self.pooling[1]) // s1 + 1
        
        self.input = in_data
        self.output = np.empty((m_samples, in_channel, out_row, out_col))
        self.max_indices = np.zeros((m_samples, in_channel, out_row, out_col),dtype=tuple)

        for m in np.arange(m_samples):
            for i in np.arange(in_channel):
                for r in np.arange(out_row):
                    for c in np.arange(out_col):
                        region = in_data[m, i, r * s0 : r*s0 + self.pooling[0], c*s1 : c*s1 + self.pooling[1]]
                        self.output[m, i, r, c] = np.max(region)
                        # this is the max indices of the region [n, i, j]. it is a np array
                        self.max_indices[m, i, r, c] = np.unravel_index(region.argmax(), region.shape)
        return self.output

    def backward(self, dz):
        m_samples,in_channel,in_row, in_col = self.input.shape
        _, out_channel, out_row, out_col = self.output.shape
        
        dx = np.zeros_like(self.input)

        for m in np.arange(m_samples):
            for i in np.arange(in_channel):
                for r in np.arange(out_row):
                    for c in np.arange(out_col):
                        dx[m, i, self.max_indices[m, i, r, c][0]+r, self.max_indices[m, i, r, c][1]+c] = dz[m, i, r, c]
        
        return dx
    
class FlattenLayer:
    def __init__(self):
        pass
    def forward(self, in_data):
        self.m_samples, self.in_channel, self.in_row, self.in_col = in_data.shape
        return in_data.reshape(self.m_samples, self.in_channel * self.in_row * self.in_col)
    def backward(self, dz):
        return dz.reshape(self.m_samples, self.in_channel, self.in_row, self.in_col)    
    
class LinearLayer:
    def __init__(self, in_num, out_num, lr = 0.001):
        self.in_num = in_num
        self.out_num = out_num
        self.w = np.random.randn(in_num, out_num)/np.sqrt(in_num)
        self.b = np.zeros((1,out_num))
        self.lr = lr

    def forward(self, in_data):
        self.input = in_data
        self.output = np.dot(in_data,self.w) + self.b
        return self.output
    def backward(self, dz):
        #similar as hidden layer in ann, but note that we do not include activation or softmax function in FCLayer.
        dw = np.dot(self.input.T, dz) 
        db = np.sum(dz,axis=0, keepdims=True)
        dx = dz.dot(self.w.T)
        self.w -= self.lr * dw
        self.b -= self.lr * db
        return dx  
    
    
class relulayer: #activation function 
    def __init__(self):
        pass
    def forward(self,in_data):
        # relu(x)= x if x>0 ; 0 otherwise
        self.input = in_data
        self.output = in_data.copy()
        self.output[self.output<0] = 0
        return self.output
    def backward(self,dz):
        dx = dz.copy()
        dx[self.input<0]=0
        return dx    
    
    
class SoftmaxLayer:
    def __init__(self):
        pass
    def softmax(self,z):
        exp_value = np.exp(z-np.amax(z, axis=1, keepdims=True)) # for stablility
        # keepdims = True means that the output's dimension is the same as of z
        softmax_scores = exp_value / np.sum(exp_value, axis=1, keepdims=True)
        return softmax_scores
    
    def forward(self, in_data):
        self.input = in_data
        self.output = self.softmax(in_data)
        return self.output
    def backward(self, y):
        return self.output - y #y_hat - y
    
    
class Network():
    def __init__(self):
        self.layers = []
        self.y_prob = None
        self.loss = None
        
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def feed_data(self, X, y):
        self.X = X
        self.y = y
    
    def feed_forward(self):                # feed forward 
        input = self.X
        for layer in self.layers:
            input = layer.forward(input)
        self.y_prob = input
        return input
    
    def back_propagation(self):                # back propagate
        dz = self.y # for the output layer, the input gradient is indeed target.
        for layer in reversed(self.layers):
            dz = layer.backward(dz) 
    
    def cross_entropy_loss(self):
        #  $L = -\sum_n\sum_{i\in C} y_{n, i}\log(\hat{y}_{n, i})$
        # calculate y_hat
        self.feed_forward()
        self.loss = -np.sum(self.y*np.log(self.y_prob + 1e-6)) 
    
    def predict(self, X_test):
        input = X_test
        for layer in self.layers:
            #if plot_feature_maps:
            #    image = (image * 255)[0, :, :]
            #    plot_sample(image, None, None)
            input = layer.forward(input)
        return np.argmax(input, axis=1)   
    
    def mini_batch_descent(self, X, y, batch_size=50):
        """
            Note that y is a 2d array. 
        """
        shuffled_idx = np.arange(X.shape[0])
        np.random.shuffle(shuffled_idx)
        n_batches = X.shape[0] // batch_size 
        
        for i in range(n_batches):
            self.feed_data(X[shuffled_idx[i*batch_size:(i+1)*batch_size]], y[shuffled_idx[i*batch_size:(i+1)*batch_size]])
            self.feed_forward()
            self.back_propagation()
        if X.shape[0] % batch_size != 0:
            self.feed_data(X[shuffled_idx[n_batches*batch_size:]], y[shuffled_idx[n_batches*batch_size:]])
            self.feed_forward()
            self.back_propagation()        
    
    

In [125]:
import time        
# load data
X_train, y_train = read_dataset('Digits_X_train.csv', 'Digits_y_train.csv')
X_test, y_test = read_dataset('Digits_X_test.csv', 'Digits_y_test.csv')

#normlize featurues / encode labels / convert features into one-hot-vector.
X_train_norm, X_test_norm = normalize_features(X_train, X_test)
y_train_ohe, y_test_ohe = ohe(y_train, y_test)
X_train_tensor,X_test_tensor = to_tensor(X_train_norm),to_tensor(X_test_norm)


# initialize network
myANN1 = Network() 

#conv;pad; for your homework
myANN1.add_layer(FlattenLayer())
#myANN.add_layer(LinearLayer(in_num=64, out_num=10,lr=0.0001))
myANN1.add_layer(LinearLayer(64,200))
myANN1.add_layer(LinearLayer(200,10))
myANN1.add_layer(relulayer())
myANN1.add_layer(SoftmaxLayer())



#start training
start = time.time()
epoch_num = 40
for i in range(epoch_num):
    myANN1.mini_batch_descent(X_train_tensor, y_train_ohe, batch_size=200)
    myANN1.cross_entropy_loss()
    y_pred = myANN1.predict(X_test_tensor)
    print('epoch = {}, current loss = {:.2f}, test accuracy = {:.2f}%'.format(i+1, myANN1.loss, 100*accuracy(y_pred, y_test.ravel())))      
    
end = time.time()
y_pred = myANN1.predict(X_test_tensor)
print('Accuracy of my ANN model is {:.2f}%'.format(accuracy(y_pred, y_test.ravel())*100))
print('Takes {:.2f}s'.format(end-start))

epoch = 1, current loss = 84.61, test accuracy = 85.11%
epoch = 2, current loss = 39.40, test accuracy = 91.33%
epoch = 3, current loss = 34.49, test accuracy = 92.22%
epoch = 4, current loss = 24.95, test accuracy = 92.44%
epoch = 5, current loss = 27.18, test accuracy = 92.89%
epoch = 6, current loss = 25.99, test accuracy = 93.78%
epoch = 7, current loss = 13.32, test accuracy = 94.00%
epoch = 8, current loss = 17.97, test accuracy = 94.22%
epoch = 9, current loss = 19.32, test accuracy = 94.00%
epoch = 10, current loss = 20.91, test accuracy = 94.44%
epoch = 11, current loss = 18.67, test accuracy = 94.67%
epoch = 12, current loss = 17.01, test accuracy = 94.67%
epoch = 13, current loss = 8.10, test accuracy = 94.44%
epoch = 14, current loss = 9.82, test accuracy = 94.89%
epoch = 15, current loss = 12.29, test accuracy = 94.67%
epoch = 16, current loss = 6.50, test accuracy = 94.44%
epoch = 17, current loss = 13.36, test accuracy = 94.44%
epoch = 18, current loss = 6.12, test accur

In [126]:
myANN2 = Network() 

#conv;pad; for your homework
myANN2.add_layer(FlattenLayer())
#myANN.add_layer(LinearLayer(in_num=64, out_num=10,lr=0.0001))
myANN2.add_layer(LinearLayer(64,100))
myANN2.add_layer(LinearLayer(100,100))
myANN2.add_layer(LinearLayer(100,10))
myANN2.add_layer(relulayer())
myANN2.add_layer(SoftmaxLayer())



#start training
start = time.time()
epoch_num = 40
for i in range(epoch_num):
    myANN2.mini_batch_descent(X_train_tensor, y_train_ohe, batch_size=200)
    myANN2.cross_entropy_loss()
    y_pred = myANN2.predict(X_test_tensor)
    print('epoch = {}, current loss = {:.2f}, test accuracy = {:.2f}%'.format(i+1, myANN2.loss, 100*accuracy(y_pred, y_test.ravel())))      
    
end = time.time()
y_pred = myANN2.predict(X_test_tensor)
print('Accuracy of my ANN model is {:.2f}%'.format(accuracy(y_pred, y_test.ravel())*100))
print('Takes {:.2f}s'.format(end-start))

epoch = 1, current loss = 76.91, test accuracy = 83.56%
epoch = 2, current loss = 41.70, test accuracy = 88.00%
epoch = 3, current loss = 20.12, test accuracy = 91.56%
epoch = 4, current loss = 23.04, test accuracy = 92.44%
epoch = 5, current loss = 10.95, test accuracy = 93.56%
epoch = 6, current loss = 16.39, test accuracy = 93.33%
epoch = 7, current loss = 16.94, test accuracy = 94.22%
epoch = 8, current loss = 12.60, test accuracy = 94.00%
epoch = 9, current loss = 10.26, test accuracy = 95.11%
epoch = 10, current loss = 11.48, test accuracy = 94.67%
epoch = 11, current loss = 11.90, test accuracy = 95.56%
epoch = 12, current loss = 9.59, test accuracy = 94.89%
epoch = 13, current loss = 9.18, test accuracy = 94.67%
epoch = 14, current loss = 7.07, test accuracy = 95.11%
epoch = 15, current loss = 7.36, test accuracy = 95.11%
epoch = 16, current loss = 7.32, test accuracy = 95.33%
epoch = 17, current loss = 7.04, test accuracy = 95.33%
epoch = 18, current loss = 6.01, test accuracy

In [127]:
myANN5 = Network() 

#conv;pad; for your homework
myANN5.add_layer(FlattenLayer())
#myANN.add_layer(LinearLayer(in_num=64, out_num=10,lr=0.0001))
myANN5.add_layer(LinearLayer(64,50))
myANN5.add_layer(LinearLayer(50,100))
myANN5.add_layer(LinearLayer(100,100))
myANN5.add_layer(LinearLayer(100,100))
myANN5.add_layer(LinearLayer(100,50))
myANN5.add_layer(LinearLayer(50,10))
myANN5.add_layer(relulayer())
myANN5.add_layer(SoftmaxLayer())



#start training
start = time.time()
epoch_num = 40
for i in range(epoch_num):
    myANN5.mini_batch_descent(X_train_tensor, y_train_ohe, batch_size=200)
    myANN5.cross_entropy_loss()
    y_pred = myANN5.predict(X_test_tensor)
    print('epoch = {}, current loss = {:.2f}, test accuracy = {:.2f}%'.format(i+1, myANN5.loss, 100*accuracy(y_pred, y_test.ravel())))      
    
end = time.time()
y_pred = myANN5.predict(X_test_tensor)
print('Accuracy of my ANN model is {:.2f}%'.format(accuracy(y_pred, y_test.ravel())*100))
print('Takes {:.2f}s'.format(end-start))

epoch = 1, current loss = 67.18, test accuracy = 82.44%
epoch = 2, current loss = 29.80, test accuracy = 91.33%
epoch = 3, current loss = 14.39, test accuracy = 93.11%
epoch = 4, current loss = 18.46, test accuracy = 94.00%
epoch = 5, current loss = 12.59, test accuracy = 92.44%
epoch = 6, current loss = 7.61, test accuracy = 93.33%
epoch = 7, current loss = 4.06, test accuracy = 92.67%
epoch = 8, current loss = 11.49, test accuracy = 94.67%
epoch = 9, current loss = 8.72, test accuracy = 94.44%
epoch = 10, current loss = 12.54, test accuracy = 95.11%
epoch = 11, current loss = 3.52, test accuracy = 94.22%
epoch = 12, current loss = 2.87, test accuracy = 94.44%
epoch = 13, current loss = 2.08, test accuracy = 95.11%
epoch = 14, current loss = 1.94, test accuracy = 94.67%
epoch = 15, current loss = 2.01, test accuracy = 95.11%
epoch = 16, current loss = 2.20, test accuracy = 94.44%
epoch = 17, current loss = 1.57, test accuracy = 94.89%
epoch = 18, current loss = 1.72, test accuracy = 9

**&#9989; **Problem 2 (60pts)****.  The MNIST datasets are given. Create a CNN with 2 convolution
layers and 2 pooling layers to classify the dataset such that:

- The first convolution layer with kernel size: (1, filters1, 5, 5), and strides = 1, padding =0.
- The second convolution layer with kernel size: (filters1, filters2, 3, 3), and strides = 1, padding = 0.
- The pooling size is (2, 2), and the stride size of pooling layers is 2.

Machine learning libraries in Python packages should NOT be used

In [167]:
import time        
# load data
X_train, y_train = read_dataset('MNIST_X_train2.csv', 'MNIST_y_train2.csv')
X_test, y_test = read_dataset('MNIST_X_test2.csv','MNIST_y_test2.csv')

#normlize featurues / encode labels / convert features into one-hot-vector.
X_train_norm, X_test_norm = normalize_features(X_train, X_test)
y_train_ohe, y_test_ohe = ohe(y_train, y_test)
X_train_tensor,X_test_tensor = to_tensor(X_train_norm),to_tensor(X_test_norm)

#print(X_train_tensor.shape)

# initialize network
myCNN = Network() 
myCNN.add_layer(ConvLayer1(lr=0.0001))
myCNN.add_layer(MaxPoolingLayer())
myCNN.add_layer(ConvLayer2(lr=0.0001))
myCNN.add_layer(MaxPoolingLayer())

#conv;pad; for your homework
myCNN.add_layer(FlattenLayer())
myCNN.add_layer(LinearLayer(in_num=50, out_num=100,lr=0.0001))
myCNN.add_layer(relulayer())
myCNN.add_layer(LinearLayer(in_num=100, out_num = y_train_ohe.shape[1],lr=0.0001))
myCNN.add_layer(SoftmaxLayer())

#start training
start = time.time()
epoch_num = 40
for i in range(epoch_num):
    myCNN.mini_batch_descent(X_train_tensor, y_train_ohe, batch_size=100)
    myCNN.cross_entropy_loss()
    y_pred = myCNN.predict(X_test_tensor)
    print('epoch = {}, current loss = {:.2f}, test accuracy = {:.2f}%'.format(i+1, myCNN.loss, 100*accuracy(y_pred, y_test.ravel())))      
    
end = time.time()
y_pred = myCNN.predict(X_test_tensor)
print('Accuracy of my CNN model is {:.2f}%'.format(accuracy(y_pred, y_test.ravel())*100))
print('Takes {:.2f}s'.format(end-start))    

epoch = 1, current loss = 176.87, test accuracy = 48.00%
epoch = 2, current loss = 122.00, test accuracy = 56.20%
epoch = 3, current loss = 120.20, test accuracy = 63.20%
epoch = 4, current loss = 90.45, test accuracy = 67.20%
epoch = 5, current loss = 96.14, test accuracy = 70.00%
epoch = 6, current loss = 70.36, test accuracy = 71.60%
epoch = 7, current loss = 91.77, test accuracy = 73.00%
epoch = 8, current loss = 90.87, test accuracy = 74.60%
epoch = 9, current loss = 79.53, test accuracy = 74.60%
epoch = 10, current loss = 79.85, test accuracy = 75.80%
epoch = 11, current loss = 75.59, test accuracy = 76.60%
epoch = 12, current loss = 69.48, test accuracy = 76.60%
epoch = 13, current loss = 66.65, test accuracy = 78.20%
epoch = 14, current loss = 53.86, test accuracy = 78.60%
epoch = 15, current loss = 66.24, test accuracy = 79.40%
epoch = 16, current loss = 82.79, test accuracy = 79.60%
epoch = 17, current loss = 71.43, test accuracy = 79.80%
epoch = 18, current loss = 71.61, tes